In [1]:
import os
import re
import math
from collections import defaultdict
import pandas as pd
import numpy as np
import json
import folium
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.display import display, HTML

# Turn slice copy warnings off
pd.options.mode.chained_assignment = None

import findspark
findspark.init()
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import min
from pyspark.sql.types import ArrayType, StringType, DoubleType, IntegerType, FloatType

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
data_folder = './data/'

In [3]:
ed_boroughs = pd.ExcelFile(data_folder + '2017-graduation_rates_public_borough.xlsx')
ed_sheet_names = ed_boroughs.sheet_names

In [4]:
ed_sheets = {}

for name in ed_sheet_names:
    df = ed_boroughs.parse(name, skiprows=list(range(7))) 
    ed_sheets[name] = df
df_all_students = ed_sheets[ed_sheet_names[1]]
    
# Show all of the columns
print(list(df_all_students))

# Show a few rows from the data
df_all_students.head()

['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Cohort', 'Total Grads', 'Unnamed: 6', 'Total Regents', 'Unnamed: 8', 'Unnamed: 9', 'Advanced Regents', 'Unnamed: 11', 'Unnamed: 12', 'Regents without Advanced', 'Unnamed: 14', 'Unnamed: 15', 'Local', 'Unnamed: 17', 'Unnamed: 18', 'Still Enrolled', 'Unnamed: 20', 'Dropout', 'Unnamed: 22', 'SACC (IEP Diploma)', 'Unnamed: 24', 'TASC (GED)', 'Unnamed: 26']


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Cohort,Total Grads,Unnamed: 6,Total Regents,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Still Enrolled,Unnamed: 20,Dropout,Unnamed: 22,SACC (IEP Diploma),Unnamed: 24,TASC (GED),Unnamed: 26
0,Borough,Category,Cohort Year,Cohort,#,#,% of cohort,#,% of cohort,% of grads,...,% of cohort,% of grads,#,% of cohort,#,% of cohort,#,% of cohort,#,% of cohort
1,Bronx,All Students,2013,4 year August,13730,9102,66.2928,8105,59.0313,89.0464,...,7.26147,10.9536,2742,19.9709,1606,11.697,118,0.859432,151,1.09978
2,Bronx,All Students,2012,4 year August,13838,8985,64.9299,8149,58.8886,90.6956,...,6.04134,9.3044,2876,20.7833,1757,12.6969,100,0.722648,110,0.794913
3,Bronx,All Students,2011,4 year August,14278,8821,61.7804,8073,56.5415,91.5202,...,5.23883,8.47976,3243,22.7133,1866,13.0691,207,1.44978,126,0.882476
4,Bronx,All Students,2010,4 year August,14229,8405,59.0695,7695,54.0797,91.5526,...,4.98981,8.44735,3418,24.0214,2013,14.1472,240,1.6867,144,1.01202


We can see that, due to overlap in naming conventions as well as slightly incompatible header formatting, the column names are not very clear and the first row is simply sub-headers.  
To clean the data up, we will remove the unecessary columns and rename the remaining columns to be more readable.  
We care primarily about the raw numbers of Total Grads as well as Dropouts.

In [5]:
df_grads_dropouts = df_all_students[['Unnamed: 0', 'Unnamed: 2', 'Cohort', 'Total Grads', 'Unnamed: 6', 'Dropout', 'Unnamed: 22']]
df_grads_dropouts.rename(columns={'Unnamed: 0': 'Borough', 'Unnamed: 2': 'Year','Unnamed: 6': 'Total Grads %', 'Unnamed: 22': 'Dropout %'}, inplace=True)
df_grads_dropouts.drop(0, inplace=True)
df_grads_dropouts.head()

,Borough,Year,Cohort,Total Grads,Total Grads %,Dropout,Dropout %
1,Bronx,2013,13730,9102,66.2928,1606,11.697
2,Bronx,2012,13838,8985,64.9299,1757,12.6969
3,Bronx,2011,14278,8821,61.7804,1866,13.0691
4,Bronx,2010,14229,8405,59.0695,2013,14.1472
5,Bronx,2009,15149,8712,57.5087,2336,15.4201


As there is not very much data, we can examine it visually to see that there are no missing values and no outliers.  
There are, though, some things to note:
- There are duplicate rows for each year. This is due to each year having cohorts that start at different times. We will use the 4 year August cohort here.
- Graduation is defined as receiving either a Regent or Local diploma from a NYC highschool - a Local diploma being easier to achieve but functionally similar.

Next, let us create a map to make the rates of graduation and dropout more visually apparent:

In [6]:
# We will pick the latest year available, 2013, for use here.

# Filtering by year 2013
df_2013 = df_grads_dropouts[df_grads_dropouts['Year'] == 2013].reset_index(drop=True)

# Selecting the 4 year August cohort
df_2013 = df_2013.iloc[::2].reset_index(drop=True)
df_2013.head()

,Borough,Year,Cohort,Total Grads,Total Grads %,Dropout,Dropout %
0,Bronx,2013,13730,9102,66.2928,1606,11.697
1,Brooklyn,2013,20096,14954,74.4128,1497,7.44924
2,Manhattan,2013,15280,11438,74.856,1117,7.31021
3,Queens,2013,19417,15112,77.8287,1238,6.37586
4,Staten Island,2013,4631,3718,80.285,276,5.95983


In [7]:
# Mapping graduation rates (%)
grads_map = folium.Map(location=[40.75, -74],
                       tiles='cartodbpositron',
                       zoom_start=11)

borough_edge = json.load(open(os.path.join('data', 'nyc-boroughs.json')))

folium.GeoJson(
    borough_edge,
    name='geojson'
).add_to(grads_map)

grads_map.choropleth(
    geo_data=borough_edge, 
    data=df_2013,
    columns=['Borough', 'Total Grads %'],
    key_on = 'feature.properties.borough',
    fill_color='BuPu', fill_opacity=0.7, line_opacity=0.3, line_color='black',
    legend_name='Graduation Percentage in NYC Boroughs, 4 Year Class of August 2013',
    highlight=True)

grads_map

In [8]:
# Mapping dropout rates (%)
drops_map = folium.Map(location=[40.75, -74],
                       tiles='cartodbpositron',
                       zoom_start=11)

borough_edge = json.load(open(os.path.join('data', 'nyc-boroughs.json')))

folium.GeoJson(
    borough_edge,
    name='geojson'
).add_to(drops_map)

drops_map.choropleth(
    geo_data=borough_edge, 
    data=df_2013,
    columns=['Borough', 'Dropout %'],
    key_on = 'feature.properties.borough',
    fill_color='BuPu', fill_opacity=0.7, line_opacity=0.3, line_color='black',
    legend_name='Dropout Percentage in NYC Boroughs, 4 Year Class of August 2013',
    highlight=True)

drops_map